<a href="https://colab.research.google.com/github/Haakam21/road-conditions/blob/master/model/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import statements
from pandas import read_excel
from array import *
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import History 
from keras.callbacks import Callback,ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
import keras.backend as K
import numpy as np
import tensorflow as tf
history = History()

#reading the data file in
my_sheet = 'Sheet 1'
file_name = './Data.xlsx'
df = read_excel(file_name, sheet_name = my_sheet)

#print data file/excel file
print(df)

#get dimensions of the data
shape = df.shape
rows = shape[0]
coloumns = shape[1] - 2

#print out the dimensions of the data that we will use
print(rows)
print(coloumns)



#make the x_training and y_training data
x_training = [[0 for x in range(coloumns)] for y in range(rows)]
y_training=[[0 for x in range(1)] for y in range(rows)]
x_trainingPandas = [[0 for x in range(coloumns)] for y in range(rows)]
y_trainingPandas =[[0 for x in range(1)] for y in range(rows)]
namesOfColoumns = ['acc_x', 'acc_y', 'acc_z', 'acc_dx', 'acc_dy', 'acc_dz', 'pothole', 'date', '(acc_z)^2', '(acc_dz)^2']

print('rows is', len(x_training))
print('coloumns is', len(x_training[0]))

#used to compute f1 score
def get_f1(y_true, y_pred): #taken from old keras source code + a website
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val


#fill up the data in x_training and y_training
#iterate over all of the data points
for ind in df.index: 
  for index in range(10):
    if index != 6 and index != 7:
      if index > 7:
        x_training[ind][index - 2] = df[namesOfColoumns[index]][ind]
        #print(x_training[ind][index - 2]) #print statements to see it run
      elif index < 6:
        x_training[ind][index] = df[namesOfColoumns[index]][ind]
        #print(x_training[ind][index]) #print statements to see it run


for ind in df.index: 
  y_training[ind][0] = df[namesOfColoumns[6]][ind]
  #print(y_training[ind][0]). #print statement to see the y values



#now that the data is loaded correctly, time to create and run the model!
# define the DL model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(12, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))


x_trainingPandas = x_training
y_trainingPandas = y_training
x_training=np.array(x_training)
y_training=np.array(y_training)

# compile the DL model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[get_f1, 'accuracy', tf.keras.metrics.AUC()]) #adam optimizer, binary crossentropy (classification) for loss function

# fit the keras model on the dataset
model.fit(x_training, y_training, epochs=300, batch_size=512, callbacks=[history])

In [ ]:
# # see the final accuracy of the model
# _, accuracy = model.evaluate(x_training, y_training)
# print('Accuracy: %.2f' % (accuracy*100))


# should write out the history of the model's performance
print(history.history.keys())
wordKeys = list(history.history.keys())
# summarize history for accuracy
plt.plot(history.history[wordKeys[2]])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history[wordKeys[0]])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
#history for f1_score
plt.plot(history.history[wordKeys[1]])
plt.title('f1_score')
plt.ylabel('f1 score')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

#history for auc
plt.plot(history.history[wordKeys[3]])
# plt.plot([0, 1, 2, 3])
plt.title('auroc')
plt.ylabel('auroc score')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()


In [ ]:
print(len(y_training))

countPotholePos = 0
countPotholeNeg = 0
for indexForPotCount in range(7100):
  if y_training[indexForPotCount][0] < 0.5:
    countPotholeNeg = countPotholeNeg + 1
  else:
    countPotholePos = countPotholePos + 1
print('number of negative potholes is', countPotholeNeg)
print('number of positive potholes is', countPotholePos)
print('Percentage of potholes data that are have a pothole', (countPotholePos)/(countPotholeNeg + countPotholePos) * 100)

In [ ]:
for layer in model.layers:
    weights = layer.get_weights()
    print(weights)

In [ ]:
from keras.losses import binary_crossentropy, categorical_crossentropy
import keras.backend as K
import numpy as np
from prettytable import PrettyTable
from prettytable import ALL
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)


# define the NEW DL model
modelF = Sequential()
modelF.add(Dense(12, input_dim=8, activation='relu'))
modelF.add(Dropout(0.4))
modelF.add(Dense(12, activation='relu'))
modelF.add(Dropout(0.4))
modelF.add(Dense(8, activation='relu'))
modelF.add(Dropout(0.4))
modelF.add(Dense(1, activation='sigmoid'))
x_training=np.array(x_training)
y_training=np.array(y_training)

# compile the DL model
modelF.compile(loss=f1_loss, optimizer='adam', metrics=[f1, 'accuracy', tf.keras.metrics.AUC()]) #adam optimizer, binary crossentropy (classification) for loss function

# fit the keras model on the dataset
modelF.fit(x_training, y_training, epochs=1000, batch_size=1024, callbacks=[history])


In [ ]:
# # see the final accuracy of the model
# _, accuracy = model.evaluate(x_training, y_training)
# print('Accuracy: %.2f' % (accuracy*100))

# should write out the history of the model's performance
print(history.history.keys())
wordKeys = list(history.history.keys())
# summarize history for accuracy
plt.plot(history.history[wordKeys[2]])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history[wordKeys[0]])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
#history for f1_score
plt.plot(history.history[wordKeys[1]])
plt.title('f1_score')
plt.ylabel('f1 score')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

#history for auc
plt.plot(history.history[wordKeys[3]])
# plt.plot([0, 1, 2, 3])
plt.title('auroc')
plt.ylabel('auroc score')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
from imblearn.over_sampling import SMOTE
#NOW TRYING THIS WITH OVERSAMPLING THE DATA
sm = SMOTE(random_state=12, ratio = 1.0)
x_res, y_res = sm.fit_sample(x_trainingPandas, y_trainingPandas)

print(len(y_res))
print(len(x_res))

countPotholePos = 0
countPotholeNeg = 0
for indexForPotCount in range(len(y_res)):
  if y_res[indexForPotCount] < 0.5:
    countPotholeNeg = countPotholeNeg + 1
  else:
    countPotholePos = countPotholePos + 1
print('number of negative potholes is', countPotholeNeg)
print('number of positive potholes is', countPotholePos)
print('Percentage of potholes data that are have a pothole', (countPotholePos)/(countPotholeNeg + countPotholePos) * 100)

In [ ]:
# define the NEW DL model
modelSample = Sequential()
modelSample.add(Dense(12, input_dim=8, activation='relu'))
modelSample.add(Dropout(0.4))
modelSample.add(Dense(12, activation='relu'))
modelSample.add(Dropout(0.4))
modelSample.add(Dense(8, activation='relu'))
modelSample.add(Dropout(0.4))
modelSample.add(Dense(1, activation='sigmoid'))

# compile the DL model
modelSample.compile(loss=f1_loss, optimizer='adam', metrics=[f1, 'accuracy', tf.keras.metrics.AUC()]) #adam optimizer, binary crossentropy (classification) for loss function

# fit the keras model on the dataset
modelSample.fit(x_res, y_res, epochs=1000, batch_size=1024, callbacks=[history])

In [ ]:
# should write out the history of the model's performance
print(history.history.keys())
wordKeys = list(history.history.keys())
# summarize history for accuracy
plt.plot(history.history[wordKeys[2]])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history[wordKeys[0]])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
#history for f1_score
plt.plot(history.history[wordKeys[1]])
plt.title('f1_score')
plt.ylabel('f1 score')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

#history for auc
plt.plot(history.history[wordKeys[3]])
# plt.plot([0, 1, 2, 3])
plt.title('auroc')
plt.ylabel('auroc score')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
# define the NEW DL model
modelAccuracy = Sequential()
modelAccuracy.add(Dense(12, input_dim=8, activation='relu'))
modelAccuracy.add(Dropout(0.4))
modelAccuracy.add(Dense(12, activation='relu'))
modelAccuracy.add(Dropout(0.4))
modelAccuracy.add(Dense(12, activation='relu'))
modelAccuracy.add(Dropout(0.4))
modelAccuracy.add(Dense(8, activation='relu'))
modelAccuracy.add(Dropout(0.4))
modelAccuracy.add(Dense(4, activation='relu'))
modelAccuracy.add(Dropout(0.4))
modelAccuracy.add(Dense(1, activation='sigmoid'))

# compile the DL model
modelAccuracy.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1, 'accuracy', tf.keras.metrics.AUC()]) #adam optimizer, binary crossentropy (classification) for loss function

# fit the keras model on the dataset
modelAccuracy.fit(x_res, y_res, epochs=400, batch_size=128, callbacks=[history])

In [ ]:
# should write out the history of the model's performance
print(history.history.keys())
wordKeys = list(history.history.keys())
# summarize history for accuracy
plt.plot(history.history[wordKeys[2]])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history[wordKeys[0]])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
#history for f1_score
plt.plot(history.history[wordKeys[1]])
plt.title('f1_score')
plt.ylabel('f1 score')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

#history for auc
plt.plot(history.history[wordKeys[3]])
# plt.plot([0, 1, 2, 3])
plt.title('auroc')
plt.ylabel('auroc score')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
#TO-DO (note for myself):
# 1. Split up data for training and validation
# 2. Compute F1 Scores
# 3. Research about how to deal with skewed data more effectively